In [1]:
!pip install -U sentence-transformers arxiv umap-learn nltk

Keyring is skipped due to an exception: 'keyring.backends'
Requirement already up-to-date: sentence-transformers in /opt/anaconda3/lib/python3.7/site-packages (2.2.2)
Requirement already up-to-date: arxiv in /opt/anaconda3/lib/python3.7/site-packages (1.4.7)
Requirement already up-to-date: umap-learn in /opt/anaconda3/lib/python3.7/site-packages (0.5.3)
Requirement already up-to-date: nltk in /opt/anaconda3/lib/python3.7/site-packages (3.8.1)


In [2]:
import arxiv
from sentence_transformers import SentenceTransformer
import umap
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mihailtereskin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [137]:
import urllib, urllib.request
from bs4 import BeautifulSoup
url = 'https://export.arxiv.org/api/query?search_query=au:robert+koenig&cat:quant-ph&start=0&max_results=59'
with urllib.request.urlopen(url) as response:
    soup = BeautifulSoup(response, "lxml")
    
print(soup.prettify())

<?xml version="1.0" encoding="UTF-8"?>
<html>
 <body>
  <feed xmlns="http://www.w3.org/2005/Atom">
   <link href="http://arxiv.org/api/query?search_query%3Dau%3Arobert%20koenig%26id_list%3D%26start%3D0%26max_results%3D59" rel="self" type="application/atom+xml"/>
   <title type="html">
    ArXiv Query: search_query=au:robert koenig&amp;id_list=&amp;start=0&amp;max_results=59
   </title>
   <id>
    http://arxiv.org/api/jzIW9LZcoAMnw+8vP6fGuLq9Ax0
   </id>
   <updated>
    2023-05-07T00:00:00-04:00
   </updated>
   <opensearch:totalresults xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">
    27762
   </opensearch:totalresults>
   <opensearch:startindex xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">
    0
   </opensearch:startindex>
   <opensearch:itemsperpage xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">
    59
   </opensearch:itemsperpage>
   <entry>
    <id>
     http://arxiv.org/abs/1304.7031v1
    </id>
    <updated>
     2013-04-25T21:47:22Z
    </update

In [168]:
titles = soup.find_all('title')
titles = list(map(lambda x:x.text, titles))
del titles[0]

titles[-1]

'Locking of accessible information and implications for the security of\n  quantum cryptography'

In [167]:
summaries = soup.find_all('summary')
summaries = list(map(lambda x:x.text, summaries))
summaries[-1]

"  The unconditional security of a quantum key distribution protocol is often\ndefined in terms of the accessible information, that is, the maximum mutual\ninformation between the distributed key S and the outcome of an optimal\nmeasurement on the adversary's (quantum) system. We show that, even if this\nquantity is small, certain parts of the key S might still be completely\ninsecure when S is used in applications, such as for one-time pad encryption.\nThis flaw is due to a locking property of the accessible information: one\nadditional (physical) bit of information might increase the accessible\ninformation by more than one bit.\n"

In [162]:
name = 'Robert Koenig'
name.split(' ', 1)[0]

'Robert'

In [173]:
def retrieve_name(name):
    split_name = name.split(' ', 1)
    return True if split_name[0] == "Robert" or split_name[1] == "Koenig" else False

names = soup.find_all('name')
names = list(map(lambda x:x.text, names))
names = list(filter(retrieve_name, names))
del names[names.index('Robert Howe')]

indices = list([names.index('Alexander Koenig'), names.index('Robert A. Bridges')])
indices

[33, 15]

In [175]:
titles_to_remove = list()
summaries_to_remove = list()

for index in indices:
    titles_to_remove.append(titles[index])
    summaries_to_remove.append(summaries[index])
    
for title, summary in zip(titles_to_remove, summaries_to_remove):
    titles.remove(title)
    summaries.remove(summary)

In [180]:
koenig_papers = list()
for title, summary in zip(titles, summaries):
    koenig_papers.append(title + '.' + '\n\n' + summary)

In [184]:
print(koenig_papers[0])

The conditional entropy power inequality for Gaussian quantum states.

  We propose a generalization of the quantum entropy power inequality involving
conditional entropies. For the special case of Gaussian states, we give a proof
based on perturbation theory for symplectic spectra. We discuss some
implications for entanglement-assisted classical communication over additive
bosonic noise channels.



In [3]:
search = arxiv.Search(query = "quantum", max_results=350, sort_by=arxiv.SortCriterion.SubmittedDate)

In [4]:
papers = list()
for result in search.results():
    papers.append(result.title + '.' + '\n\n' + result.summary)

In [5]:
print(papers[0])

Electronic Excited States from a Variance-Based Contracted Quantum Eigensolver.

Electronic excited states of molecules are central to many physical and
chemical processes, and yet they are typically more difficult to compute than
ground states. In this paper we leverage the advantages of quantum computers to
develop an algorithm for the highly accurate calculation of excited states. We
solve a contracted Schr\"odinger equation (CSE) -- a contraction (projection)
of the Schr\"odinger equation onto the space of two electrons -- whose
solutions correspond identically to the ground and excited states of the
Schr\"odinger equation. While recent quantum algorithms for solving the CSE,
known as contracted quantum eigensolvers (CQE), have focused on ground states,
we develop a CQE based on the variance that is designed to optimize rapidly to
a ground or excited state. We apply the algorithm in a classical simulation
without noise to computing the ground and excited states of H$_{4}$ and BH.


In [6]:
#tokenize text into sentences
tokenized_papers = [nltk.sent_tokenize(paper) for paper in papers]

In [7]:
print(tokenized_papers[0])

['Electronic Excited States from a Variance-Based Contracted Quantum Eigensolver.', 'Electronic excited states of molecules are central to many physical and\nchemical processes, and yet they are typically more difficult to compute than\nground states.', 'In this paper we leverage the advantages of quantum computers to\ndevelop an algorithm for the highly accurate calculation of excited states.', 'We\nsolve a contracted Schr\\"odinger equation (CSE) -- a contraction (projection)\nof the Schr\\"odinger equation onto the space of two electrons -- whose\nsolutions correspond identically to the ground and excited states of the\nSchr\\"odinger equation.', 'While recent quantum algorithms for solving the CSE,\nknown as contracted quantum eigensolvers (CQE), have focused on ground states,\nwe develop a CQE based on the variance that is designed to optimize rapidly to\na ground or excited state.', 'We apply the algorithm in a classical simulation\nwithout noise to computing the ground and excit

In [8]:
embeddings = list()
model = SentenceTransformer('all-MiniLM-L6-v2')
for token_paper in tokenized_papers:
    embeddings.append(model.encode(token_paper))

In [26]:
import numpy as np
np.argmin([embedding.shape[0] for embedding in embeddings])
tokenized_papers[335]

['Treatment-Response Analysis of Tumor as A Quantum Particle.',
 'In this article, I present a novel and computational-efficient approach for\ntreatment-response modeling of tumor progression-free survival (PFS)\nprobability using the physical phenomenon of a quantum particle walking on a\none-dimensional lattice with the presence of a proximate trap.']

In [30]:
reducer = umap.UMAP(n_neighbors=2, init='random', n_components=2)
umap_embeddings=list(map(lambda x:reducer.fit_transform(x), embeddings))

/opt/anaconda3/lib/python3.7/site-packages/umap/umap_.py:2345: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  "n_neighbors is larger than the dataset size; truncating to "
/opt/anaconda3/lib/python3.7/site-packages/umap/umap_.py:133: UserWarning: A large number of your vertices were disconnected from the manifold.
Disconnection_distance = inf has removed 0 edges.
It has fully disconnected 2 vertices.
You might consider using find_disconnected_points() to find and remove these points from your data.
Use umap.utils.disconnected_vertices() to identify them.
  f"A large number of your vertices were disconnected from the manifold.\n"


ValueError: zero-size array to reduction operation maximum which has no identity